#Running on OS X 10.14 w/Python3.7. I encountered the same above errors, but was able to get past them by #upgrading cython:

pip3 install -U cython

#removing the following 2 lines from setup.cfg:

[sdist]
pre-hook.sdist_pre_hook = guidedlda._setup_hooks.sdist_pre_hook

#and then running the original installation instructions:

git clone https://github.com/vi3k6i5/GuidedLDA
cd GuidedLDA
sh build_dist.sh
python setup.py sdist
pip3 install -e .

In [21]:
!pip install wheel

In [26]:
!find guidedlda -name '*.pyx' -exec cython {} \;

In [30]:
import numpy as np
import guidedlda

https://github.com/vi3k6i5/guidedlda

#input below, X, is a document-term matrix

In [31]:
X = guidedlda.datasets.load_data(guidedlda.datasets.NYT)
vocab = guidedlda.datasets.load_vocab(guidedlda.datasets.NYT)
word2id = dict((v, idx) for idx, v in enumerate(vocab))

In [34]:
X.shape

(8447, 3012)

In [35]:
X.sum()

1221626

In [36]:
# Normal LDA without seeding
model = guidedlda.GuidedLDA(n_topics=5, n_iter=100, random_state=7, refresh=20)
model.fit(X)

INFO:guidedlda:n_documents: 8447
INFO:guidedlda:vocab_size: 3012
INFO:guidedlda:n_words: 1221626
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -11489265
INFO:guidedlda:<20> log likelihood: -9844667
INFO:guidedlda:<40> log likelihood: -9694223
INFO:guidedlda:<60> log likelihood: -9642506
INFO:guidedlda:<80> log likelihood: -9617962
INFO:guidedlda:<99> log likelihood: -9604031


In [37]:
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: company percent market business plan pay price increase
Topic 1: game play team win player season second start
Topic 2: life child write man school woman father family
Topic 3: place open small house music turn large play
Topic 4: official state government political states issue leader case


In [38]:
# Guided LDA with seed topics.
seed_topic_list = [['game', 'team', 'win', 'player', 'season', 'second', 'victory'],
                   ['percent', 'company', 'market', 'price', 'sell', 'business', 'stock', 'share'],
                   ['music', 'write', 'art', 'book', 'world', 'film'],
                   ['political', 'government', 'leader', 'official', 'state', 'country', 'american','case', 'law', 'police', 'charge', 'officer', 'kill', 'arrest', 'lawyer']]


In [40]:
model = guidedlda.GuidedLDA(n_topics=5, n_iter=100, random_state=7, refresh=20)

In [41]:
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

In [42]:
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:guidedlda:n_documents: 8447
INFO:guidedlda:vocab_size: 3012
INFO:guidedlda:n_words: 1221626
INFO:guidedlda:n_topics: 5
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -11486372
INFO:guidedlda:<20> log likelihood: -9765300
INFO:guidedlda:<40> log likelihood: -9659651
INFO:guidedlda:<60> log likelihood: -9622617
INFO:guidedlda:<80> log likelihood: -9604295
INFO:guidedlda:<99> log likelihood: -9592268


In [44]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: game play team win season player second point start victory
Topic 1: company percent market price business sell pay plan executive buy
Topic 2: play life man music place turn book woman write thing
Topic 3: official government state political leader states issue member case country
Topic 4: school child city family problem student life program group state


In [45]:
doc_topic = model.transform(X)
for i in range(9):
    print("top topic: {} Document: {}".format(doc_topic[i].argmax(), ', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[0:5]])))

top topic: 4 Document: plant, increase, food, increasingly, animal
top topic: 3 Document: explain, life, country, citizen, nation
top topic: 2 Document: thing, solve, problem, machine, carry
top topic: 2 Document: company, authority, opera, artistic, director
top topic: 4 Document: wife, rape, husband, file, state
top topic: 3 Document: partner, lawyer, attorney, client, indict
top topic: 2 Document: roll, place, soon, treat, rating
top topic: 3 Document: city, drug, program, commission, report
top topic: 1 Document: company, comic, series, case, executive


Saving the model

In [ ]:
from six.moves import cPickle as pickle
with open('guidedlda_model.pickle', 'wb') as file_handle:
    pickle.dump(model, file_handle)
# load the model for prediction
with open('guidedlda_model.pickle', 'rb') as file_handle:
    model = pickle.load(file_handle)
doc_topic = model.transform(X)